# Evaluation using the BeFree corpus 

### EUADR dataset
The EU-ADR dataset contains annotations on drugs, diseases, genes and proteins, and associations between them. In this study, we used only GDAs to evaluate the method. Each association is classified according to its level of certainty as positive association (PA), negative association (NA), speculative association (SA); or false association (FA). The EU-ADR corpus is based on 100 MEDLINE abstracts for each association set, and its annotation was conducted by three experts. 


#  -------------------------------------------------------------------------------------------

# imports

In [1]:
import tensorflow as tf
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras_tqdm import TQDMNotebookCallback
import numpy as np
np.random.seed(1337)
from keras_tqdm import TQDMNotebookCallback
import nltk
import xml.etree.ElementTree as ET
import pandas as pd
import os
import string
from nltk.tokenize import TreebankWordTokenizer
from numpy.random import random_sample
import re
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from keras.layers import Embedding, Flatten,LSTM
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation,  Input, merge,Conv1D,MaxPooling1D,GlobalMaxPooling1D,Convolution1D
from keras import regularizers
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from keras.layers import Concatenate, concatenate
from keras import backend as K
from keras.layers import multiply
from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
random_seed=1337

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/raidmax/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


###  Define Callback functions to generate Mesures

In [2]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# Experiments to reproduce the results of Table 8 

### Load Prerocssed Data

In [51]:
with open('../data/pickles/befree_EUADR_2class_wikipedia-pubmed-and-PMC.pickle', 'rb') as handle:        
    
    W_train = pickle.load(handle)
    d1_train = pickle.load(handle)
    d2_train = pickle.load(handle)
    Y_train = pickle.load(handle)
    Tr_word_list = pickle.load(handle)
    word_vectors = pickle.load(handle)
    word_dict = pickle.load(handle)
    d1_dict = pickle.load(handle)
    d2_dict = pickle.load(handle)
    label_dict = pickle.load(handle)
    MAX_SEQUENCE_LENGTH = pickle.load(handle)


### Create Position Embedding Vectors

In [52]:
import keras
from keras_pos_embd import TrigPosEmbedding

model = keras.models.Sequential()
model.add(TrigPosEmbedding(
    input_shape=(None,),
    output_dim=20,                      # The dimension of embeddings.
    mode=TrigPosEmbedding.MODE_EXPAND,  # Use `expand` mode
    name='Pos-Embd',
))
model.compile('adam', keras.losses.mae, {})
model.summary()

d1_train_embedded=model.predict(d1_train)

d1_train_embedded.shape

d2_train_embedded=model.predict(d2_train)

d2_train_embedded.shape

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Pos-Embd (TrigPosEmbedding)  (None, None, 20)          0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


(355, 102, 20)

### Prepare Word Embedding Layer

In [53]:
EMBEDDING_DIM=word_vectors.shape[1]
embedding_matrix=word_vectors


param='binary'

def create_embedding_layer(l2_reg=0.01,use_pretrained=True,is_trainable=False):
    
    if use_pretrained:        
        return Embedding(len(word_dict) ,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=is_trainable,embeddings_regularizer=regularizers.l2(l2_reg))
    
    else:    
        return Embedding(len(word_dict) ,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH)
            
    

### Prepare Attention Mechanism

In [54]:
INPUT_DIM = 2
TIME_STEPS = MAX_SEQUENCE_LENGTH


In [55]:
def attentionNew(inputs):
    inputs = Lambda(lambda x: tf.keras.backend.tanh(x))(inputs)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = multiply([inputs, a_probs])
    output_attention_mul = Lambda(lambda x: tf.keras.backend.tanh(x))(output_attention_mul)
    return output_attention_mul


### Create the Model

In [59]:
dropRate=0.1
param='binary'
def build_model_positionAttention():    
    tf.set_random_seed(1337)
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedding_layer=create_embedding_layer(use_pretrained=True,is_trainable=False)
    embedded_sequences = embedding_layer(sequence_input)

    
    pos_embedd_1=Input(shape=(MAX_SEQUENCE_LENGTH,20), dtype='float32')
    pos_embedd_2=Input(shape=(MAX_SEQUENCE_LENGTH,20), dtype='float32')
    
    embedded_sequences = concatenate([embedded_sequences,pos_embedd_1,pos_embedd_2])
    x= Conv1D(64,2, activation='relu')(embedded_sequences)
    x = MaxPooling1D(3)(x)
    x = Dropout(dropRate)(x)
    conv_sequence_7=GlobalMaxPooling1D()(x)    #x = Flatten()(x)



    
    #bilstm=keras.layers.Bidirectional (LSTM(MAX_SEQUENCE_LENGTH,recurrent_dropout=0.05,return_sequences=True), merge_mode='avg', weights=None)
    #attInp=bilstm(embedded_sequences)    
    #attentionForward=attentionNew(attInp)
    #lstm_sequence=Flatten()(attentionForward)
    
    #merge = concatenate([conv_sequence_w3,conv_sequence_w4,conv_sequence_w5,lstm_sequence])
    
    
    x = Dense(16,activation= keras.activations.relu, kernel_regularizer=regularizers.l2(0.05))(conv_sequence_7)
    
    x = Dropout(dropRate)(x)

    preds = Dense(2, activation='softmax')(x)
    model = Model(inputs=[sequence_input, pos_embedd_1,pos_embedd_2],outputs=preds)
    opt=keras.optimizers.Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['acc',f1])
    
    #model.summary()
    return model
    

### Run the Evaluation using 10 fold Cross Validation  

In [60]:
def train_and_evaluate_model_lstm_position():
    prec_list=[]; reca_list=[]; fscore_list=[] ; fold=0
    skf = StratifiedKFold(n_splits=10, random_state=random_seed)
    Y = [np.argmax(y, axis=None, out=None) for y in Y_train]
    all_histories=[]
    for train_index, test_index in skf.split(W_train,Y):     
        fold+=1
        X_train, X_test = W_train[train_index], W_train[test_index] 
        pos_train1, pos_test1 = d1_train_embedded[train_index], d1_train_embedded[test_index] 
        pos_train2, pos_test2 = d2_train_embedded[train_index], d2_train_embedded[test_index] 
        y_train, y_test = Y_train[train_index], Y_train[test_index]
        model = None # Clearing the NN.
        #model = build_model()
        model = build_model_positionAttention()
        earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)        

        #history=model.fit([X_train,pos_train1,pos_train2], y_train, validation_data=([X_test,pos_test1,pos_test2],y_test) ,epochs=10, batch_size=32,verbose=0,callbacks=[TQDMNotebookCallback()])
        history=model.fit([X_train,pos_train1,pos_train2], y_train, validation_data=([X_test,pos_test1,pos_test2],y_test) ,epochs=50, batch_size=32,verbose=0,callbacks=[TQDMNotebookCallback(),earlystopper])
        #history=model.fit(X_train, y_train, validation_data=(X_test,y_test) ,epochs=5, batch_size=32,verbose=0,callbacks=[TQDMNotebookCallback(),earlystopper])
        all_histories.append(history)
        predicted = np.argmax(model.predict([X_test,pos_test1,pos_test2]), axis=1)
        
        #indices=is_not_negative[test_index]       
        #negative_indices=np.nonzero(indices==False)[0]
        #predicted[negative_indices]=1
        
        y_test_to_label = np.argmax(y_test, axis=1)
        prec, reca, fscore, sup = precision_recall_fscore_support(y_test_to_label, predicted, average=param)
        #print(classification_report(y_test_to_label,predicted))

        print("Fold {:d}: Precision:{:.2f}% Recall:{:.2f}% Fscore:{:.2f}% ".format(fold,prec*100, reca*100, fscore*100))
        prec_list.append(prec), reca_list.append(reca) , fscore_list.append(fscore)
        #K.clear_session()
        #return history
    precission=sum(prec_list)/len(prec_list)*100 
    recall=sum(reca_list)/len(reca_list)*100 
    fscore=sum(fscore_list)/len(fscore_list)*100    
    print("Final: Precision:{:.2f}% Recall:{:.2f}% Fscore:{:.2f}% ".format(precission, recall, fscore))
    return all_histories

In [61]:
all_histories=train_and_evaluate_model_lstm_position()

Fold 1: Precision:72.97% Recall:100.00% Fscore:84.37% 


Fold 2: Precision:72.97% Recall:100.00% Fscore:84.37% 


Fold 3: Precision:74.29% Recall:100.00% Fscore:85.25% 


Fold 4: Precision:76.47% Recall:100.00% Fscore:86.67% 


Fold 5: Precision:100.00% Recall:76.92% Fscore:86.96% 


Fold 6: Precision:78.79% Recall:100.00% Fscore:88.14% 


Fold 7: Precision:75.76% Recall:96.15% Fscore:84.75% 


Fold 8: Precision:73.53% Recall:96.15% Fscore:83.33% 


Fold 9: Precision:74.29% Recall:100.00% Fscore:85.25% 


Fold 10: Precision:74.29% Recall:100.00% Fscore:85.25% 
Final: Precision:77.33% Recall:96.92% Fscore:85.43% 
